<a href="https://colab.research.google.com/github/MrCherveN/Machine-Learning--4-0_HomeTasks/blob/main/HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Python/Machine Learning для людей  4.0/Модуль 2. Навчання з учителем/Тема 2.2 Задача класифікації. Логістична регресія. Принцип роботи та методи оцінки якості/data/customer_segmentation_train.csv')

In [4]:
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [6]:
df.isnull().sum()

,0
ID,0
Gender,0
Ever_Married,140
Age,0
Graduated,78
Profession,124
Work_Experience,829
Spending_Score,0
Family_Size,335
Var_1,76


In [7]:
df.Segmentation.unique()

array(['D', 'A', 'B', 'C'], dtype=object)

In [8]:
segmentation_codes = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
df['Segmentation'] = df.Segmentation.map(segmentation_codes)

train_df, test_df = train_test_split(df, test_size=0.20, random_state=42, stratify=df['Segmentation'])

target_col = 'Segmentation'
input_cols = list(train_df.columns)[1:-1]

train_inputs, train_targets = train_df[input_cols].copy(), train_df[target_col].copy()
test_inputs, test_targets = test_df[input_cols].copy(), test_df[target_col].copy()


numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
print(numeric_cols)
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()
print(categorical_cols)

imputer_num = SimpleImputer(strategy = 'median').fit(train_inputs[numeric_cols])
train_inputs[numeric_cols] = imputer_num.transform(train_inputs[numeric_cols])
test_inputs[numeric_cols] = imputer_num.transform(test_inputs[numeric_cols])


imputer_cat = SimpleImputer(strategy = 'most_frequent').fit(train_inputs[categorical_cols])
train_inputs[categorical_cols] = imputer_cat.transform(train_inputs[categorical_cols])
test_inputs[categorical_cols] = imputer_cat.transform(test_inputs[categorical_cols])


scaler = StandardScaler().fit(train_inputs[numeric_cols])
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
test_inputs[numeric_cols] = scaler.transform(test_inputs[numeric_cols])


encoder = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore').fit(train_inputs[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))
train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
test_inputs[encoded_cols] = encoder.transform(test_inputs[categorical_cols])

cols_for_model = numeric_cols + encoded_cols

['Age', 'Work_Experience', 'Family_Size']
['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [9]:
train_inputs.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Gender_Male,...,Spending_Score_Average,Spending_Score_High,Spending_Score_Low,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
917,Female,No,-0.695320,Yes,Artist,1.970880,Low,-1.231118,Cat_6,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3398,Male,Yes,1.703982,Yes,Entertainment,-0.456381,Average,-0.564314,Cat_6,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2045,Female,No,-0.635337,Yes,Entertainment,-0.456381,Low,0.769294,Cat_6,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8060,Female,Yes,0.264401,Yes,Artist,-0.759788,Average,2.102901,Cat_6,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4604,Female,Yes,-0.935250,No,Doctor,1.970880,Low,-1.231118,Cat_7,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
cat_index = [i for i, col in enumerate(train_inputs.columns) if col in categorical_cols]

In [11]:
train_inputs_smotenc = train_df[input_cols].copy()

In [12]:

train_inputs_smotenc[numeric_cols] = imputer_num.transform(train_inputs_smotenc[numeric_cols])
train_inputs_smotenc[categorical_cols] = imputer_cat.transform(train_inputs_smotenc[categorical_cols])
train_inputs_smotenc[numeric_cols] = scaler.transform(train_inputs_smotenc[numeric_cols])

smotenc = SMOTENC(random_state=42,  categorical_features=cat_index)
train_inputs_smotenc, train_targets_smotenc = smotenc.fit_resample(train_inputs_smotenc, train_targets)

train_inputs_smotenc[encoded_cols] = encoder.transform(train_inputs_smotenc[categorical_cols])

smotetomek = SMOTETomek(random_state=42)
train_inputs_smotetomek, train_targets_smotetomek = smotetomek.fit_resample(train_inputs[cols_for_model], train_targets)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [13]:
# Логістична регресія зі стратегією one-vs-rest (OvR) на оигінальних даних
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(train_inputs[cols_for_model], train_targets)
ovr_predictions = ovr_model.predict(test_inputs[cols_for_model])

print(f' Метрики precision та recall для кожного класу на тестовому наборі - на оигінальних даних \n {classification_report(test_targets, ovr_predictions)}')


#на збалансованих даних з  SMOTENC
log_reg_smotenc = LogisticRegression(solver='liblinear')
ovr_model_smotenc = OneVsRestClassifier(log_reg_smotenc)
ovr_model_smotenc.fit(train_inputs_smotenc[cols_for_model], train_targets_smotenc)
ovr_predictions_smotenc = ovr_model_smotenc.predict(test_inputs[cols_for_model])

print(f' Метрики precision та recall для кожного класу на тестовому наборі - на  SMOTENC \n {classification_report(test_targets, ovr_predictions_smotenc)}')



#на збалансованих даних з SMOTE-Tomek

log_reg_smotetomek = LogisticRegression(solver='liblinear')
ovr_model_smotetomek = OneVsRestClassifier(log_reg_smotetomek)
ovr_model_smotetomek.fit(train_inputs_smotetomek[cols_for_model], train_targets_smotetomek)
ovr_predictions_smotetomek = ovr_model_smotetomek.predict(test_inputs[cols_for_model])

print(f' Метрики precision та recall для кожного класу на тестовому наборі - на SMOTE-Tomek \n {classification_report(test_targets, ovr_predictions_smotetomek)}')


 Метрики precision та recall для кожного класу на тестовому наборі - на оигінальних даних 
               precision    recall  f1-score   support

           0       0.42      0.45      0.43       394
           1       0.41      0.18      0.25       372
           2       0.49      0.61      0.54       394
           3       0.65      0.76      0.70       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.50      0.51      0.49      1614

 Метрики precision та recall для кожного класу на тестовому наборі - на  SMOTENC 
               precision    recall  f1-score   support

           0       0.42      0.48      0.45       394
           1       0.40      0.25      0.31       372
           2       0.51      0.59      0.54       394
           3       0.68      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted a

***Висновки:***

- Я обрав метрику f1-score macro avg, бо вона даэ усереднене значення гармоныйного середнього по всым класам.
- Трішки краще себе показала модель  SMOTENC.
- Різниці дійсно несуттєві. Можливо, наявних незалежних ознак недостатньо для чіткого розмежування записів на класи.